# **This notebook contains example on the usage of the class ``StructureFactor``**:
The data should be load as an object of type ``PointPattern``. This can be done using the calss ``PointPattern``. 

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#%load_ext autotime  # must be added to dependencies
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ``HomogeneousPoissonPointProcess``:
The class ``HomogeneousPoissonPointProcess`` allows to sample a Poisson point process using ``generate_sample`` in a box window using ``BoxWindow`` of ``spatial_windows`` or a in a ball window using ``BallWindow``of ``spatial_windows``.

In [ ]:
from structure_factor.homogeneous_poisson_process import HomogeneousPoissonPointProcess
from structure_factor.spatial_windows import BoxWindow

L_poisson = 50 #length side of the cube containing the Poisson point process
bounds = np.array([[-L_poisson/2, -L_poisson/2], [L_poisson/2, L_poisson/2]]) #bounds of the box window containing the point process
window = BoxWindow(bounds) #box window containing the point process

intensity_poisson = 3 #intensity of the Poisson point process
process = HomogeneousPoissonPointProcess(intensity=intensity_poisson) 
points_1 = process.generate_sample(window=window) #Poisson point process in the cubic window

plt.plot(points_1[:,0], points_1[:,1], 'b,')
plt.show()

## ``PointPattern``:
The class ``StructureFactor``take an object of type ``PointPattern``.
The following is an exmaple of tansfering the sample of the Poisson point proces ``points_1`` to an objects of type ``PointPattern``.
The objects of type ``PointPattern`` have 3 attributes "points", "window" and "intensity". 

In [ ]:
from structure_factor.point_pattern import PointPattern
poisson_data_1 = PointPattern(points_1, window, intensity=intensity_poisson) #making the poisson point proces an object of type Point Pattern

Generating a Poisson point process in a disk using ``HomogeneousPoissonPointProcess``with ``BallWindow``.

In [ ]:
from structure_factor.spatial_windows import BallWindow
R_poisson = 70 #radius of the disk containing the Poisson point process
center_poisson = [0,0] #center of the disk contaniing the Poisson point process

window = BallWindow(center_poisson, R_poisson) #creating ball window
process = HomogeneousPoissonPointProcess() 
points_2 = process.generate_sample(window=window) #Poisson point process in the ball window

plt.plot(points_2[:,0], points_2[:,1], 'b,')
plt.show()

## ``PointPattern.restrict_to_cubic_window``:
The method ``compoute_scattering_intensity`` of the class ``StructureFactor`` works for ``PointPattern`` with **cubic** window. So we create the method ``restrict_to_cubic_window`` to restrict the PointPattern to a cubic window for the perpose of use in ``compoute_scattering_intensity``.
The following is an example of transforming the poisson point process ``points``to an object of type ``PointPattern`` restricted to a cubic window using the method ``restrict_to_cubic_window``of the class ``PointPattern``.

In [ ]:
Poisson_data_2 = PointPattern(points_2) #to an object of type PointPattern

L_poisson_2 = int(R_poisson/(np.sqrt(2))) #length of the cubic window 
poisson_data_2 = Poisson_data_2.restrict_to_cubic_window(x_min= -L_poisson_2/2, y_min=-L_poisson_2/2, L=L_poisson_2 ) #restrict to a cubic window

plt.plot(poisson_data_2.points[:,0], poisson_data_2.points[:,1], 'b,')
plt.show()

# loading data from the saved file "test_data.dat":
Poisson point process ("poisson_data") , Ginibre point pross ("ginibre_data") , lattice $\mathbb{Z}^2$ ("z_2_data") and matching processus of Michael Andreas Klatt, Günter Last, D. Yogeshwaran that we will denoted by kly ("kly_data") defined in https://arxiv.org/abs/1810.00265

In [ ]:
import pickle
path_data = "../data"
with open(os.path.join(path_data, "test_data.dat"), "rb") as data:
    poisson_data, ginibre_data, kly_data, z_2_data = pickle.load(data, encoding="bytes")

The data in "test_data.dat": "poisson_data", "ginibre_data", "kly_data" and "z_2_data",  are objects of type ``PointPattern`` so they have 3 attributes "points", "window" and "intensity". 
For example ``poisson_data.points`` are the points of a Poisson point process  and ``poisson_data.window`` is the window containing the points, and ``poisson_data.intensity`` is the intensity of the point process.
``poisson_data.points``, ``kly_data.points`` and ``z_2_data.points`` lie in "BoxWindow", to see the bound of the window use for exmaple ``poisson_data.window.bounds``. while ``ginibre_data.points`` lie in a "BallWindow", to see the center and raduis use ``ginibre_data.window.center`` resp. ``ginibre_data.window.raduis``.

In [ ]:
print("poisson_data : point", poisson_data.points.shape , ", intensity=", poisson_data.intensity)
print("window of poisson_data :bounds ",  kly_data.window.bounds )
print("window of ginibre_data: center ",ginibre_data.window.center, ", raduis=", ginibre_data.window.radius)

#  ``StructureFactor``:

In [ ]:
from structure_factor.structure_factor_new import StructureFactor

In [ ]:
sf_poisson= StructureFactor(poisson_data)
sf_ginibre = StructureFactor(ginibre_data)
sf_kly = StructureFactor(kly_data)
sf_z_2 = StructureFactor(z_2_data)

## ``compute_sf_via_scattering_inetensity``:

### for Poisson point process: 
we know that the pair correlation function and the structure factor of the Poisson point process are equal to 1 so we always plot the line  $𝑦=1$ as a theoretical reference to the Poisson point process 

In [ ]:
wave_length_poisson, scattering_intensity_poisson = sf_poisson.compute_sf_via_scattering_intensity(max_k=15, meshgrid_size=None)

### ``plot_scattering_intensity``
The method ``plot_scattering_intensity`` take the output of the method ``compute_sf_via_scattering_intensity`` to plot them.
The optional argument ``binning_parameter``correspnding to the parameters used to average the values of the scattering intensity over bins, and ``exact_sf`` is the sctructure factor function if it's known.

In [ ]:
sf_poisson.plot_scattering_intensity(wave_length_poisson, scattering_intensity_poisson, bins=40)

### For Ginibre point process:
as the Ginibre point process lies in a ball window we will use ``restrict_to_cubic_window`` to restricting it into a cubic window.

In [ ]:
plt.plot(ginibre_data.points[:,0], ginibre_data.points[:,1], 'b,')
plt.show()

In [ ]:
R_ginibre = ginibre_data.window.radius
L_ginibre = R_ginibre/np.sqrt(2)
ginibre_data_restricted = ginibre_data.restrict_to_cubic_window(x_min=-L_ginibre/2, y_min=-L_ginibre/2, L=L_ginibre)

sf_ginibre_2 = StructureFactor(ginibre_data_restricted)

plt.plot(ginibre_data_restricted.points[:,0], ginibre_data_restricted.points[:,1], 'b,')
plt.show()

In [ ]:
wave_length_ginibre, scattering_intensity_ginibre = sf_ginibre_2.compute_sf_via_scattering_intensity(max_k =10, meshgrid_size=300)

In [ ]:
n = np.floor(10 * L_ginibre / (2 * np.pi))
wave_length_ginibre.shape

In [ ]:
wave_length_ginibre[int(n),int(n)]

In [ ]:
arr = np.arange(12) 
print(arr)
print(np.argwhere( arr==111))

In [ ]:
mask = np.ones(len(arr), dtype=bool)
print(mask)

In [ ]:
arr = np.arange(12) +3
index_zero= np.argwhere(arr==0)
mask = np.ones(len(arr), dtype=bool)
mask[index_zero] = False
result = arr[mask,...]
print(result)

In [ ]:
arr = np.arange(-5,5, 11)
np.arg
mask = np.ones(len(arr), dtype=bool)
mask[[0]] = False
result = arr[mask,...]
print(mask)

In [ ]:
exact_sf_ginibre = lambda x : 1 - np.exp(-x**2/4) # exact structure factor for the Ginibre point process

sf_ginibre_2.plot_scattering_intensity(wave_length_ginibre, scattering_intensity_ginibre, plot_type="all", exact_sf=exact_sf_ginibre, bins=40 )

# for the processus of  Michael Andreas Klatt, Günter Last, D. Yogeshwaran that we will denoted by kly defined in https://arxiv.org/abs/1810.00265

In [ ]:
norm_k_kly, si_kly = sf_kly.compute_scattering_intensity(L=300, maximum_k=20)

In [ ]:
sf_kly.plot_scattering_intensity(norm_k_kly, si_kly, plot_type="plot", bins=40 )

In [ ]:
norm_k_kly_, si_kly_ = sf_kly.compute_scattering_intensity(L=300, maximum_k=10, meshgrid_size=150)

In [ ]:
sf_kly.plot_scattering_intensity(norm_k_kly_, si_kly_, plot_type="all", bins=40)

# for a lattice $\mathbb{Z}^2$

In [ ]:
norm_k_z2, si_z2 = sf_z_2.compute_scattering_intensity(L=300, maximum_k=50)

In [ ]:
sf_z_2.plot_scattering_intensity(norm_k_z2, si_z2, plot_type="plot", bins=40 )

In [ ]:
plt.plot(norm_k_z2, si_z2, 'b.')

In [ ]:
norm_k_z2_, si_z2_ = sf_z_2.compute_scattering_intensity(L=300, maximum_k=20, meshgrid_size=150)

In [ ]:
sf_z_2.plot_scattering_intensity(norm_k_z2_, si_z2_, plot_type="all", bins=40)